In [28]:
import pandas as pd

selected_search_prompt = 2
selecred_extraction_prompt = 5

page_information = pd.read_csv('./data/page_information.csv')
page_information["query"] = page_information["query"].apply(lambda x: x.replace(" -cyberleninka -youtube -видео -pdf", ""))
queries = pd.read_csv('./data/web_search_results.csv')

queries = queries[queries["prompt"] == selected_search_prompt]
topics = pd.read_csv('./data/extraction_results.csv')
topics = topics[topics["prompt"] == selecred_extraction_prompt]

In [29]:
page_information['word_count'] = page_information["data"].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
page_information['word_count'].describe().astype(int)

count       480
mean       1169
std        6412
min           3
25%         136
50%         633
75%        1148
max      139150
Name: word_count, dtype: int32

In [5]:
page_information[page_information["word_count"] < 15]

,query,url,data,word_count
4,топ 5 упражнений для утренней зарядки видео,https://alphamed.by/utrennyaya-zaryadka-kak-pr...,{'error': 'Failed to retrieve the page. Status...,9
16,кейсы применения big data для оптимизации бизн...,https://habr.com/ru/companies/newprolab/articl...,{'title': '7 кейсов использования технологий B...,14
17,как анализ больших данных повышает эффективнос...,https://networking.camp/useful/advice/big-data,{'error': 'Failed to retrieve the page. Status...,9
24,методы развития критического мышления с упражн...,https://practicum.yandex.ru/blog/priemy-razvit...,"{'title': 'Ð\x92Ñ\x8b Ð½Ðµ Ñ\x80Ð¾Ð±Ð¾Ñ\x82?',...",6
31,методы повышения продуктивности и организованн...,https://sbercrm.com/blog/business/tpost/tkcjl9...,{'title': 'Как повысить продуктивность работы ...,10
...,...,...,...,...
461,Влияние цифровых технологий на доступность обр...,https://www.ppfsvfu.ru/jour/article/download/2...,"{'title': 'No title found', 'paragraphs': []}",6
463,Практические советы для снижения тревоги и упр...,https://gorzdrav.org/blog/kak-borotsja-so-stre...,"{'title': 'Access Blocked', 'paragraphs': []}",5
466,Как развить лидерские качества?,https://www.sravni.ru/text/kak-razvit-v-sebe-l...,{'error': 'Failed to retrieve the page. Status...,9
467,Методы развития лидерских качеств для начинающих,https://gershanovich.ru/blog/effektivnye-upraz...,{'error': 'Failed to retrieve the page. Status...,9


In [6]:
import os

def get_models(directory, extension=".gguf"):
    return [filename.split(".")[0] for filename in os.listdir(directory) if filename.endswith(extension)]

models = get_models("../models")
print(models)

['gemma-2-9b-it-Q8_0_L', 'saiga-llama-3-q8_0', 'suzume-llama-3-8b-Q8_0']


In [7]:
from langchain_community.llms import LlamaCpp
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


llama3_tokens = {
    'first_token': '<|begin_of_text|>',
    'start_header': '<|start_header_id|>', 
    'end_header': '<|end_header_id|>', 
    'end_message': '<|eot_id|>'
}
gemma2_tokens = {
    'first_token': '',
    'start_header': '<start_of_turn>',
    'end_header': '',
    'end_message': '<end_of_turn>'
}

def get_llm(path):

    model = LlamaCpp(
        model_path=path,
        temperature=0,
        n_ctx=8192,
        max_tokens=2000,
        top_p=1,
        repeat_penalty=1.1,
        # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        verbose=False,
        n_gpu_layers=33
    )
    return model

def get_chat(model_name):
    chat = ChatOllama(model=model_name, temperature=0)
    return chat

In [138]:
summarization_prompt_base = """{first_token}{start_header}system{end_header}
Выдели из данного текста основные факты и данные, которые имеют наибольшую ценность и значимость. Особое внимание удели информации, касающейся темы: «{topic}», но не исключай другие важные факты и данные. Исключи общие фразы, лишние детали и несущественную информацию. 
Сформулируй результаты в виде краткого списка ключевых пунктов.

Текст:
{page}

Используй русский язык для формирования результатов.

{end_message}{start_header}assistant{end_header}\n"""

summarization_prompt_candidate_1 = """
Выдели из данного текста ключевые факты и данные, которые имеют наибольшую значимость. Особое внимание удели информации, касающейся темы: «{topic}», но не исключай другие важные факты и данные. Исключи общие фразы, повторяющуюся информацию и любые несущественные детали. 
Представь результаты в виде списка из коротких и информативных пунктов. Под ключевыми фактами подразумеваются даты, цифры, имена, места, конкретные события и статистические данные.

Текст:
{page}

Используй русский язык для формирования результатов.

{end_message}{start_header}assistant{end_header}\n"""

summarization_prompt_candidate_2 = """
Проанализируй данный текст и выдели из него ключевые факты и данные, которые имеют наибольшую значимость. Особое внимание удели информации, касающейся темы: «{topic}», но не исключай другие важные факты и данные. Исключи общие фразы, повторяющуюся информацию и любые несущественные детали. 
Представь результаты в виде списка из коротких и информативных пунктов. 

Под ключевыми фактами подразумеваются:
- Даты и временные промежутки
- Числовые данные и статистика
- Имена людей и организаций
- Названия мест
- Конкретные события и факты

Текст:
{page}

Используй русский язык для формирования результатов без дополнительных объяснений.

{end_message}{start_header}assistant{end_header}\n"""
summarization_candidates = [
    summarization_prompt_base,
    summarization_prompt_candidate_1,
    summarization_prompt_candidate_2
]

In [13]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate

import pandas as pd
import tqdm
from langchain_core.exceptions import OutputParserException


def get_summarization_prompt(prompt):
    summarization_prompt = PromptTemplate(
        template=prompt,
        input_variables=["first_token", "start_header", "end_header", "end_message", "thema", "page"],
    )
    return summarization_prompt

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_len(text):
    return len(text.split())

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=get_len,
    is_separator_regex=False,
)

def split_chunk(chunk, max_words=1000):
    chunks = text_splitter.create_documents([chunk])
    return chunks

def combine_chunks(chunks, max_words=1000):
    combined_chunks = []
    current_chunk = ""

    for chunk in chunks:
        words_in_chunk = chunk.split()
        current_chunk_words = current_chunk.split()
        
        if len(current_chunk_words) + len(words_in_chunk) <= max_words:
            if current_chunk:
                current_chunk += " " + chunk
            else:
                current_chunk = chunk
        else:
            if current_chunk:
                combined_chunks.append(current_chunk)
            current_chunk = chunk
            
            if len(words_in_chunk) > max_words:
                split_chunks = split_chunk(chunk, max_words)
                combined_chunks.extend(split_chunks)
                current_chunk = ""
            else:
                current_chunk = chunk

    if current_chunk:
        combined_chunks.append(current_chunk)

    return combined_chunks

In [25]:
from langchain_core.output_parsers import StrOutputParser

num_message = 0
model = models[0]
input_dict = gemma2_tokens.copy() if "gemma" in model else llama3_tokens.copy()

topic = topics[(topics["num_message"] == num_message) & (topics["model"] == model)]["Тема"].iloc[0]

prompt = summarization_candidates[0]
summarization_prompt = get_summarization_prompt(prompt)
llm = get_chat(model)
summary_chain = summarization_prompt | llm | StrOutputParser()


# Invoke

In [84]:
def summarization(content):
    input_dict["topic"] = topic
    input_dict["page"] = content
    summary = summary_chain.invoke(input_dict)
    return summary

def page_summarization(page):
    page_chunks = page.get("paragraphs", None)
    
    chunks = combine_chunks([" ".join(chunk.split()) for chunk in page_chunks if len(chunk.split()) > 3], max_words=max_words)
    
    chunks_summaries = [summarization(chunk) for chunk in chunks]
    while len(chunks_summaries) > 1:
        chunks_summaries = [summarization(chunk) for chunk in combine_chunks(chunks_summaries, max_words=max_words)]
    return chunks_summaries[0]

def query_summarization(query_pages):
    query_pages = [eval(p) for p in query_pages]
    pages_summary = [page_summarization(page) for page in query_pages if page.get("paragraphs", None)]
    if len(pages_summary) > 1:
        summary = [summarization(chunk) for chunk in combine_chunks(pages_summary, max_words=max_words)]
        return summary
    return pages_summary[0]

In [85]:
%%time

query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 500
s = query_summarization(pages.data.tolist())

CPU times: total: 297 ms
Wall time: 2min 10s


In [77]:
%%time
query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 1000
s = query_summarization(pages.data.tolist())

CPU times: total: 93.8 ms
Wall time: 1min 16s


In [80]:
print(s[0])

## Ключевые факты и данные из текста:

**Для увеличения количества прочитанных книг:**

* Составьте wishread list с 8 темами развития и минимум 20 книгами в каждой, включая книги успешных людей.
* Используйте психологический прием: представьте, что у вас осталось полгода жизни - какие 10 книг вы бы выбрали?
* Определите цель на год (например, 25 или 100 книг).
* Установите ежедневный минимальный объем чтения (начать с 20 страниц в день).
* Запланируйте время для чтения в своем ежедневнике.

**Дополнительные советы:**

* Работа в книжном издательстве предоставляет много возможностей для чтения и доступа к библиотеке.
* Замените другие занятия чтением (телевизор, интернет, соцсети).
* Читайте рассказы - они отнимают меньше времени.
* Не делайте перерывов между книгами: начните новую книгу в тот же день, когда закончили предыдущую.

**Список книг для вдохновения и расширения кругозора:** 

* **Отсутствует в тексте.**





In [87]:
%%time
query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 2000
s = query_summarization(pages.data.tolist())

CPU times: total: 93.8 ms
Wall time: 34.6 s


In [88]:
print(s[0])

## Ключевые пункты текста:

**Преимущества чтения:**

* Расширение знаний, понимания мира и эмоционального интеллекта.
* Развитие критического мышления и формирования собственных мнений.
* Получение удовольствия от эскапизма, развлечений и новых перспектив.
* Нахождение вдохновения, мотивации и руководства в жизни.

**Рекомендации по чтению:**

*  Читать 15-30 минут ежедневно для достижения значительных результатов.
*  Использовать аромамасла (апельсин, эвкалипт, гвоздика) для концентрации.
*  Отнестись к книге как к мудрому учителю, читая с открытым умом и желанием учиться.
*  Применить технику с метрономом для быстрого чтения без отвлечений.

**Важно:** 

* Текст не содержит конкретного списка книг для расширения кругозора и поиска вдохновения.





In [89]:
%%time
query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 4000
s = query_summarization(pages.data.tolist())

CPU times: total: 109 ms
Wall time: 22.2 s


In [90]:
print(s[0])

Here are the key points from the text, focusing on reading for personal growth and inspiration:

* **Reading Benefits:** Reading expands knowledge, understanding, emotional intelligence, critical thinking, and helps form opinions. It also offers enjoyment, escape, and inspiration.
* **Reading Goals:** Aim for 15-30 minutes of daily reading to potentially read numerous books per year.
* **Habit Formation Tips:**  Be selective with books, optimize your reading environment, prioritize reading time, set goals, and find your preferred reading style.
* **Active Reading:** Engage with the text by highlighting, taking notes, and reflecting on what you read for deeper understanding.


The text emphasizes that reading is a valuable habit for personal growth and inspiration, offering practical advice on how to make it a regular part of life. 



# Batch page chunks

In [94]:
def summarization(content):
    input_dict["topic"] = topic
    input_dict["page"] = content
    summary = summary_chain.invoke(input_dict)
    return summary

def page_summarization(page):
    page_chunks = page.get("paragraphs", None)
    
    chunks = combine_chunks([" ".join(chunk.split()) for chunk in page_chunks if len(chunk.split()) > 3], max_words=max_words)

    batch = []
    for chunk in chunks:
        input_dict["topic"] = topic
        input_dict["page"] = chunk
        batch.append(input_dict)

    chunks_summaries = summary_chain.batch(batch)
    
    # chunks_summaries = [summarization(chunk) for chunk in chunks]
    while len(chunks_summaries) > 1:
        chunks_summaries = [summarization(chunk) for chunk in combine_chunks(chunks_summaries, max_words=max_words)]
    return chunks_summaries[0]

def query_summarization(query_pages):
    query_pages = [eval(p) for p in query_pages]
    pages_summary = [page_summarization(page) for page in query_pages if page.get("paragraphs", None)]
    if len(pages_summary) > 1:
        summary = [summarization(chunk) for chunk in combine_chunks(pages_summary, max_words=max_words)]
        return summary
    return pages_summary[0]

In [93]:
%%time
query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 1500
s = query_summarization(pages.data.tolist())

CPU times: total: 78.1 ms
Wall time: 36.8 s


# Batch queries chunks

In [210]:
def query_summarization(query_pages):
    query_pages = [eval(p) for p in query_pages]
    num_iters = 0
    chunks_batch = []
    for page in query_pages:
        page_chunks = page.get("paragraphs", None)
        if page_chunks:
            chunks = [" ".join(chunk.split()) for chunk in page_chunks if len(chunk.split()) > 3]
            if len(chunks) == 0:
                continue
            chunks_batch.extend(chunks)
    while (len(chunks_batch) > 1) | (num_iters == 4):
        num_iters += 1
        chunks_batch = combine_chunks(chunks_batch, max_words=max_words)
        batch = []
        for chunk in chunks_batch:
            input_dict["topic"] = topic
            input_dict["page"] = chunk
            batch.append(input_dict)

        chunks_batch = summary_chain.batch(batch)
    return chunks_batch        

In [127]:
%%time
query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 2000
s = query_summarization(pages.data.tolist())

CPU times: total: 15.6 ms
Wall time: 9.31 s


In [128]:
s

'This text appears to be a promotional piece for an English language learning program, possibly with some elements about the benefits of reading.  Here\'s a breakdown:\n\n**Key Points:**\n\n* **Emphasis on Reading:** The text highlights the importance of reading for personal development and knowledge acquisition. It encourages active reading techniques like making notes and asking questions.\n* **Reading Goals:** It suggests realistic reading goals, starting with 15-30 minutes daily and increasing to an hour if possible.  It also provides a rough estimate of how many books one could read in a year based on these timeframes.\n* **Strategies for Habit Formation:** The text mentions strategies for making reading a habit, though it doesn\'t elaborate on them.\n* **Call to Action:** There are multiple calls to action throughout the text, encouraging readers to contact the program for more information or to sign up.\n\n**Promotional Elements:**\n\n* **Testimonials/Benefits:** While not expli

In [132]:
%%time

prompt = summarization_candidates[1]
summarization_prompt = get_summarization_prompt(prompt)
# llm = get_chat(model)
summary_chain = summarization_prompt | llm | StrOutputParser()

query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 2000
s = query_summarization(pages.data.tolist())

CPU times: total: 15.6 ms
Wall time: 8.6 s


In [134]:
print(s)

Текст, который ты предоставил, представляет собой фрагмент статьи о пользе чтения и о том, как развить эту привычку.  В нем рассказывается:

* **О важности активного чтения:** делать пометки, выписывать цитаты и задавать себе вопросы помогает лучше усвоить информацию.
* **О необходимости разнообразия в чтении:** не бояться пробовать новые авторов, жанры и темы.
* **О времени, которое нужно уделять чтению:** даже 15-30 минут в день могут быть полезны, а час - позволит прочитать больше книг.
* **О количестве книг, которые можно прочитать за год:**  в среднем это может быть от 12 до 30 книг, в зависимости от времени, которое человек уделяет чтению.
* **О стратегиях для развития привычки чтения:** статья обещает дать конкретные советы по этому вопросу.


В тексте также упоминаются темы:

* Рейтинг лучших книг о финансовой грамотности по возрасту.
*  Особенности изучения английского языка через чтение сказок в оригинале. 
* Причины, почему дети не любят читать и как это исправить.



Текст 

In [139]:
%%time

prompt = summarization_candidates[2]
summarization_prompt = get_summarization_prompt(prompt)
# llm = get_chat(model)
summary_chain = summarization_prompt | llm | StrOutputParser()

query = "сколько книг читать в год для саморазвития"
pages = page_information[page_information["query"] == query].drop_duplicates()
max_words = 2000
s = query_summarization(pages.data.tolist())

CPU times: total: 0 ns
Wall time: 9.18 s


In [140]:
print(s)

Чтение - это бесконечный процесс саморазвития. Не бойтесь знакомиться с новыми авторами, жанрами и темами. Помните, что самое главное – это наслаждаться процессом и открывать для себя неизведанные миры на страницах книг. 

Время, которое рекомендуется выделить на чтение, варьируется в зависимости от режима дня и занятости. Однако 15-30 минут в день уже станут приемлемым вариантом. Средняя скорость чтения взрослого человека составляет около 200-300 слов в минуту. За 15 минут он прочитает порядка 3 000-4 500 слов, что соответствует примерно 10-15 страницам. Таким образом, даже при ограниченном времени, можно прочитать определенное количество книг. Если делать акцент на чтение в свободное время, к примеру вечером перед сном, то выделите около часа на прочтение выбранного произведения. Это уже открывает возможности для освоения большего количества литературы. 

Одна книга, в среднем, содержит около 300-400 страниц. При условии, что человек читает 10 страниц в день, то за месяц он осилит 1 

In [194]:
get_models("../models")

['saiga-llama-3-q8_0', 'suzume-llama-3-8b-Q8_0']

In [195]:
results = pd.DataFrame()
max_words = 2000

for model in get_models("../models")[1:]:
    print(model)
    llm = get_chat(model)
    input_dict = gemma2_tokens.copy() if "gemma" in model else llama3_tokens.copy()
    for candidate in summarization_candidates:
        summarization_prompt = get_summarization_prompt(candidate)
        summary_chain = summarization_prompt | llm | StrOutputParser()
        outputs = {}
        inputs = queries[queries["model"] == model].reset_index(drop=True)
        for num, row in tqdm.tqdm(inputs.iterrows(), total=40):
            
            for query in eval(row.queries):
                pages = page_information[page_information["query"] == query].drop_duplicates()
                summary = query_summarization(pages.data.tolist())
                if len(summary) == 1:
                    summary = summary[0]
                output = {
                    "summary": summary,
                    "query": query
                }
                for key in output.keys():
                    outputs[key] = outputs.get(key, []) + [output[key]]
        
        summarization_df = pd.DataFrame.from_dict(outputs)
        summarization_df['stage'] = "web_search" # bug
        summarization_df['prompt'] = summarization_candidates.index(candidate)
        summarization_df['model'] = model.split("/")[-1]
        results = pd.concat([results, summarization_df], axis=0)
        results.to_csv('./data/summarization_results.csv', index=False)

saiga-llama-3-q8_0


 32%|█████████████████████████▋                                                     | 13/40 [39:24<1:21:51, 181.92s/it]


KeyboardInterrupt: 

Модели архитектуры llama3 не справляется с длинной контекста и начинают генерировать много текста. Работает только до 1 000 слов, но тогда теряется часть смысла и увеличивается компьют. Пайплайн не успеет отработать за менее 1 минуты

In [235]:
results = pd.read_csv('./data/summarization_results.csv')

In [237]:
results["word_count"] = results["summary"].apply(lambda x: len(x.split()) if isinstance(x, str) else len(x))

In [238]:
results

,summary,query,stage,prompt,model,word_count
0,## Ключевые факты и данные:\n\n* **Расширение ...,книги для расширения кругозора и вдохновения,web_search,0,gemma-2-9b-it-Q8_0_L,88
1,## Ключевые факты и данные:\n\n**Скорочтение:*...,топ 5-7 книг для саморазвития и поиска вдохнов...,web_search,0,gemma-2-9b-it-Q8_0_L,104
2,## Ключевые факты и данные из текста:\n\n**Чте...,сколько книг читать в год для саморазвития,web_search,0,gemma-2-9b-it-Q8_0_L,128
3,Текст отлично структурирован и содержит много ...,как развить привычку читать и выбрать интересн...,web_search,0,gemma-2-9b-it-Q8_0_L,154
4,[],топ 5 упражнений для утренней зарядки видео,web_search,0,gemma-2-9b-it-Q8_0_L,1
...,...,...,...,...,...,...
235,- Писатель Айодеджи Авосика.\n- Стивен Кови.\n...,как улучшить коммуникативные навыки,web_search,2,gemma-2-9b-it-Q8_0_L,11
236,- 179 человек в Великобритании заразились виру...,как путешествия меняют человека,web_search,2,gemma-2-9b-it-Q8_0_L,19
237,- Для путешествий в горы необходимы знания и у...,влияние путешествий на личность и саморазвитие,web_search,2,gemma-2-9b-it-Q8_0_L,61
238,"Пожалуйста, предоставьте текст для анализа. \n",практические методы развития самодисциплины,web_search,2,gemma-2-9b-it-Q8_0_L,5


In [240]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["word_count"]].mean()

,,word_count
prompt,model,
0,gemma-2-9b-it-Q8_0_L,132.1875
1,gemma-2-9b-it-Q8_0_L,135.8750
2,gemma-2-9b-it-Q8_0_L,76.6500


In [242]:
results[results["word_count"] < 20]

,summary,query,stage,prompt,model,word_count
4,[],топ 5 упражнений для утренней зарядки видео,web_search,0,gemma-2-9b-it-Q8_0_L,1
16,[],кейсы применения big data для оптимизации бизн...,web_search,0,gemma-2-9b-it-Q8_0_L,1
21,"Пожалуйста, предоставьте текст, из которого ну...",#ОбщественноеМнение #СоциальныеСети #Медиа,web_search,0,gemma-2-9b-it-Q8_0_L,17
24,[],методы развития критического мышления с упражн...,web_search,0,gemma-2-9b-it-Q8_0_L,1
31,"Извини, но я не понимаю, что ты спрашиваешь. П...",методы повышения продуктивности и организованн...,web_search,0,gemma-2-9b-it-Q8_0_L,15
38,[],как составить бюджет для начинающих,web_search,0,gemma-2-9b-it-Q8_0_L,1
84,[],топ 5 упражнений для утренней зарядки видео,web_search,1,gemma-2-9b-it-Q8_0_L,1
96,[],кейсы применения big data для оптимизации бизн...,web_search,1,gemma-2-9b-it-Q8_0_L,1
104,[],методы развития критического мышления с упражн...,web_search,1,gemma-2-9b-it-Q8_0_L,1
111,"Извини, но я не понимаю, что ты спрашиваешь. П...",методы повышения продуктивности и организованн...,web_search,1,gemma-2-9b-it-Q8_0_L,15


In [245]:
queries.queries.iloc[0]

"['книги для расширения кругозора и вдохновения', 'топ 5-7 книг для саморазвития и поиска вдохновения']"

In [249]:
for sample in results[results["query"] == 'книги для расширения кругозора и вдохновения'].summary.tolist():
    print(sample)
    print("=============")

## Ключевые факты и данные:

* **Расширение тематики МИФ:**  Книги о путешествиях, кулинарии, саморазвитии, отношениях и языках стали популярны помимо бизнес-литературы. 
* **Популярность научпопа:** 
    * Книги о науке и математике обрели широкую аудиторию после успеха "Удовольствия от Х".
    * "Голая статистика" стала хитом, признанным Афишей лучшим научпопом лета.
    * Исследовательские книги об отношениях ("Размножение в неволе", "Как хочет женщина") вошли в топы продаж.
* **Будущее за научпопом:**  Авторы считают, что научно-популярная литература имеет большое будущее.
* **Бесплатная книга:** До 17 июля можно скачать книгу "Мозг во сне" бесплатно.



Вот ключевые факты и данные из текста:

* **2015 год:** Вышла книга "Удовольствие от Х", которая показала популярность книг о науке и математике.
* **"Голая статистика":** Научпоп-книга, признанная Афишей лучшим изданием лета.
* **"Размножение в неволе" и "Как хочет женщина":**  Исследовательские книги об отношениях, которые стали 

In [250]:
for sample in results[results["query"] == 'топ 5-7 книг для саморазвития и поиска вдохновения'].summary.tolist():
    print(sample)
    print("=============")

## Ключевые факты и данные:

**Скорочтение:**

*  Развивает память, мышление, внимательность.
*  Позволяет быстрее читать, понимать и запоминать текст.
*  Применяется в учебе, работе, подготовке к встречам/презентациям.
*  Тренировка индивидуальна, с учетом уровня развития каждого ученика.

**Обучение скорочтению:**

*  В Академии Skills предлагают занятия для детей с 4 лет и взрослых.
*  Есть бесплатный пробный урок.
*  Методика основана на многолетнем опыте и учитывает эмоциональное состояние ребенка.

**Книги:**

*  Не являются основным инструментом обучения скорочтению, но могут служить дополнительной поддержкой и мотивацией.


**Детская одаренность:**

*  Книга посвящена диагностике и развитию детской одаренности в современной образовательной среде.
*  Направлена на педагогов, психологов, студентов педагогических вузов. 




## Ключевые факты и данные:

**Скорочтение:**

* Методика скорочтения сокращает время чтения в 10 раз.
* Индивидуальный подход к обучению, программа подбирает